Notebook prepared by Henrique Lopes Cardoso (hlc@fe.up.pt).

# PREPROCESSING

## Tokenization

*Tokenization* is the process of spliting an input text into tokens (words or other relevant elements, such as punctuation).

#### Making use of regular expressions

We can tokenize a piece of text by using a regular expression tokenizer, such as the one available in **NLTK**.

For starters, let's stick to alphanumerical sequences of characters.

In [2]:
!pip install nltk
import nltk
from nltk import regexp_tokenize

text = 'That U.S.A. poster-print costs $12.40...'

pattern = '[a-zA-Z0-9_]+'
tokens = regexp_tokenize(text, pattern)
print(len(tokens))
print(tokens)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
9
['That', 'U', 'S', 'A', 'poster', 'print', 'costs', '12', '40']


We can refine the regular expression to obtain a more sensible tokenization.

In [3]:
pattern = r'''(?x)           # set flag to allow verbose regexps
        (?:[A-Z]\.)+         # abbreviations, e.g. U.S.A.
        | \w+(?:-\w+)*       # words with optional internal hyphens
        | \$?\d+(?:\.\d+)?%? # currency and percentages, e.g. $12.40, 82%
        | \.\.\.             # ellipsis
        | [][.,;"'?():-_`]   # these are separate tokens; includes ], [
        '''

tokens = regexp_tokenize(text, pattern)
print(len(tokens))
print(tokens)

6
['That', 'U.S.A.', 'poster-print', 'costs', '$12.40', '...']


#### Using NLTK

NLTK also includes a word tokenizer, which gets roughly the same result (it finds "words" and punctuation).

In [6]:
nltk.download('punkt')
nltk.download('punkt_tab')


from nltk import word_tokenize

text = 'That U.S.A. poster-print costs $12.40...'
tokens = word_tokenize(text)

print(len(tokens))
print(tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davideteixeira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/davideteixeira/nltk_data...


7
['That', 'U.S.A.', 'poster-print', 'costs', '$', '12.40', '...']


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
word_tokenize("I don't think we're flying today.")

['I', 'do', "n't", 'think', 'we', "'re", 'flying', 'today', '.']

You can try [other tokenizers](https://www.nltk.org/api/nltk.tokenize.html) available in NLTK.

In [8]:
# try out the wordpunct tokenizer


Let's get a sentence from the user and tokenize it.

In [9]:
import os

s = input("Enter some text:")
tokens = word_tokenize(s)

print("You typed", len(tokens), "words:", tokens)

Enter some text:Hello, I'm Davide Teixeira
You typed 6 words: ['Hello', ',', 'I', "'m", 'Davide', 'Teixeira']


#### Sentence segmentation

We may also be interested in spliting the text into sentences.

In [10]:
from nltk import sent_tokenize

text = "Hello. Are you Mr. Smith? Just to let you know that I have finished my M.Sc. and Ph.D. on AI. I loved it!"
sentences = sent_tokenize(text)

print(sentences)
print("Number of sentences:", len(sentences))

['Hello.', 'Are you Mr. Smith?', 'Just to let you know that I have finished my M.Sc.', 'and Ph.D. on AI.', 'I loved it!']
Number of sentences: 5


#### Experimenting with long texts

We can try downloading a book from [Project Gutenberg](https://www.gutenberg.org/).

In [11]:
from urllib import request

url = "http://www.gutenberg.org/files/2554/2554-0.txt"
response = request.urlopen(url)
raw = response.read().decode('utf8')

print(len(raw))
print(raw[:75])

1135214
*** START OF THE PROJECT GUTENBERG EBOOK 2554 ***




CRIME AND PUNISHMENT



How many sentences are there? Printout the second sentence (index 1).

In [13]:
sentences = sent_tokenize(raw)

print("Number of sentences:", len(sentences))

Number of sentences: 11940


How many tokens are there? What is the index of the first token in the second sentence?

In [16]:
# insert your code here
tokens = word_tokenize(raw)
print("Number of tokens:", len(tokens))

Number of tokens: 253688


And how many types (unique words) are there? Which is the most frequent one? *(Hint: use a [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) container from collections.)*

In [21]:
# insert your code here
from collections import Counter
c = Counter(tokens)
print("Number of different tokens:", len(c))

Number of different tokens: 11103


#### Dealing with multi-word expressions (MWE)

Sometimes we want certain words to stick together when tokenizing, such as in multi-word names.

In [22]:
word_tokenize("Good muffins cost $3.88\nin New York.")

['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.']

One way to do it is to suply our own lexicon and make use of NLTK's [MWE tokenizer](https://www.nltk.org/api/nltk.tokenize.mwe.html).

In [23]:
from nltk.tokenize import MWETokenizer
from nltk import sent_tokenize, word_tokenize

s = "Good muffins cost $3.88\nin New York."
mwe = MWETokenizer([('New', 'York'), ('Hong', 'Kong')], separator=' ')

[mwe.tokenize(word_tokenize(sent)) for sent in sent_tokenize(s)]

[['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New York', '.']]

Try out your own multi-word expressions to tokenize text.

In [24]:
# try out your own multi-word expressions
s = "Hi, my name is Davide. Hello, World!."
mwe = MWETokenizer([('Davide', 'Davide'), ('World', 'Everyone')], separator=' ')

[mwe.tokenize(word_tokenize(sent)) for sent in sent_tokenize(s)]

[['Hi', ',', 'my', 'name', 'is', 'Davide', '.'],
 ['Hello', ',', 'World', '!', '.']]

### Tokenization in large language models

Tokenization in large language models is more involved, and usually consists of training a vocabulary on a large corpus, using algorithms such as Byte-Pair Encoding (BPE), WordPiece, or Unigram.

OpenAI models use mostly BPE tokenizers, made available in the [tiktoken](https://github.com/openai/tiktoken) library, which you can also explore [here](https://tiktokenizer.vercel.app/).

The [SentencePiece](https://github.com/google/sentencepiece) library is another alternative.

A very nice introduction to GPT tokenizers can be found [here](https://www.youtube.com/watch?v=zduSFxRajkE).

In [26]:
!pip install tiktoken
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # check the available tokenizers at https://tiktokenizer.vercel.app/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [27]:
s = "Hellow world, how are you doing? NLP rocks!"
token_ids = enc.encode(s)

In [28]:
token_ids

[39, 5412, 1917, 11, 1268, 527, 499, 3815, 30, 452, 12852, 23902, 0]

In [29]:
enc.decode(token_ids)

'Hellow world, how are you doing? NLP rocks!'

Check out what the token ids correspond to by iterating and decoding each id at a time.

In [30]:
# insert your code here
# Don't know here

## Stemming and Lemmatization

*Stemming* and *Lemmatization* are techniques used to normalize tokens, so as to reduce the size of the vocabulary.
Whereas lemmatization is a process of finding the root of the word, stemming typically applies a set of transformation rules that aim to cut off word final affixes.

#### Stemming

NLTK includes one of the most well-known stemmers: the [Porter stemmer](https://www.emerald.com/insight/content/doi/10.1108/00330330610681286/full/pdf?casa_token=eT_IPtH_eLEAAAAA:Z3lAtxWdxf0FL479mL-A7tC-_QRzxNeeyC2DFLyWwGBlcj6DQcwu2Bnq37waDPcXKOnXkMMDtKGyCaYGZtYcb3lgBZ9uaHKUNO0JCMivSdPE4HTe).

In [31]:
from nltk.stem import PorterStemmer

# initialize the Porter Stemmer
porter = PorterStemmer()

Let's use an illustrative piece of text:

In [32]:
sentence = '''The European Commission has funded a numerical study to analyze the purchase of a pipe organ with no noise
for Europe's organization. Numerous donations have followed the analysis after a noisy debate.'''

# tokenize: split the text into words
word_list = nltk.word_tokenize(sentence)

print("\nOriginal word list:", word_list)
print("\nOriginal number of distinct tokens:", len(set(word_list)))


Original word list: ['The', 'European', 'Commission', 'has', 'funded', 'a', 'numerical', 'study', 'to', 'analyze', 'the', 'purchase', 'of', 'a', 'pipe', 'organ', 'with', 'no', 'noise', 'for', 'Europe', "'s", 'organization', '.', 'Numerous', 'donations', 'have', 'followed', 'the', 'analysis', 'after', 'a', 'noisy', 'debate', '.']

Original number of distinct tokens: 31


Now, we stem the tokens in the text:

In [33]:
# stem list of words and join
stemmed_output = ' '.join([porter.stem(w) for w in word_list])
print("Stemmed text:", stemmed_output)

# tokenize: split the text into words
stemmed_word_list = nltk.word_tokenize(stemmed_output)

print("\nStemmed word list:", stemmed_word_list)
print("\nStemmed number of distinct tokens:", len(set(stemmed_word_list)))

Stemmed text: the european commiss ha fund a numer studi to analyz the purchas of a pipe organ with no nois for europ 's organ . numer donat have follow the analysi after a noisi debat .

Stemmed word list: ['the', 'european', 'commiss', 'ha', 'fund', 'a', 'numer', 'studi', 'to', 'analyz', 'the', 'purchas', 'of', 'a', 'pipe', 'organ', 'with', 'no', 'nois', 'for', 'europ', "'s", 'organ', '.', 'numer', 'donat', 'have', 'follow', 'the', 'analysi', 'after', 'a', 'noisi', 'debat', '.']

Stemmed number of distinct tokens: 28


You can see the reduced vocabulary size. Some tokens are over-generalized (semantically different tokens that get the same stem), while others are under-generalized (semantically similar tokens that get different stems).

Try out [other stemmers](https://www.nltk.org/api/nltk.stem.html) available in NLTK.

In [ ]:
# try out other stemmers
# there are a lot

We can try a few for Portuguese:

In [35]:
# Portuguese stemmer: https://www.nltk.org/_modules/nltk/stem/rslp.html
from nltk.stem import RSLPStemmer

nltk.download('rslp')

stemmer = RSLPStemmer()
sentence = "Estou mesmo a gostar desta unidade curricular, todos gostamos de unidades curriculares interessantes."

word_list = nltk.word_tokenize(sentence)
stemmed_output = ' '.join([stemmer.stem(w) for w in word_list])
print(stemmed_output)

[nltk_data] Downloading package rslp to
[nltk_data]     /Users/davideteixeira/nltk_data...


est mesm a gost dest unidad curricul , tod gost de unidad curricul interess .


[nltk_data]   Unzipping stemmers/rslp.zip.


In [36]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("portuguese")
sentence = "Estou mesmo a gostar desta unidade curricular, todos gostamos de unidades curriculares interessantes."

word_list = nltk.word_tokenize(sentence)
stemmed_output = ' '.join([stemmer.stem(w) for w in word_list])
print(stemmed_output)

estou mesm a gost dest unidad curricul , tod gost de unidad curricul interess .


#### Lemmatization

NLTK includes a [lemmatizer based on WordNet](https://www.nltk.org/api/nltk.stem.wordnet.html).

In [38]:
# WordNet lemmatizer
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

sentence = "Men and women love to study artificial intelligence while studying data science. Don't you? My feet and teeth are clean!"


# tokenize: Split the sentence into words
word_list = nltk.word_tokenize(sentence)
print(word_list)

# lemmatize list of words
lemmatized_output = [lemmatizer.lemmatize(w) for w in word_list]
print(lemmatized_output)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/davideteixeira/nltk_data...


['Men', 'and', 'women', 'love', 'to', 'study', 'artificial', 'intelligence', 'while', 'studying', 'data', 'science', '.', 'Do', "n't", 'you', '?', 'My', 'feet', 'and', 'teeth', 'are', 'clean', '!']
['Men', 'and', 'woman', 'love', 'to', 'study', 'artificial', 'intelligence', 'while', 'studying', 'data', 'science', '.', 'Do', "n't", 'you', '?', 'My', 'foot', 'and', 'teeth', 'are', 'clean', '!']


Compare the result with stemming applied to the same text.

In [39]:
# compare with stemming


## spaCy

SpaCy includes several [language processing pipelines](https://spacy.io/usage/processing-pipelines) that streamline several NLP tasks at once. We can use one of the available [trained pipelines](https://spacy.io/models).

In [48]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy validate


import spacy
nlp = spacy.load("en_core_web_sm")


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
zsh:1: command not found: python
/opt/homebrew/opt/python@3.13/bin/python3.13: No module named spacy


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

We simply pass the sentence through the language processing pipeline (in this case, for English):

In [44]:
sent = nlp(sentence)
print(sent)
print(len(sent))

NameError: name 'nlp' is not defined

As you can see, we now have a sequence of tokens, each of which has specific [attributes](https://spacy.io/api/token#attributes) attached. For instance, we can easily get the lemma for each word:

In [ ]:
for token in sent:
    print(token.text, token.lemma_)